In [1]:
#MUST RUN
#import code

import pandas as pd
import numpy as np
import math
import random
import collections
import itertools
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

In [2]:
#MUST RUN
#establishes deck of cards

suits = ["s", "h", "d", "c"]
cards = ["2","3","4","5","6","7","8","9","T","J","Q","K","A"]
deck = [cards[i] + suits[j] for j in range(len(suits)) for i in range(len(cards)) ]

In [3]:
#MUST RUN
#establishes functions to determine hand strength

def check_quads(num_counts, numbers_to_values):

    score = 7
    score += numbers_to_values[num_counts.most_common()[0][0]]        
    score += numbers_to_values[num_counts.most_common()[1][0]]/100

    return score

def check_full_house(num_counts, numbers_to_values):

    score = 6
    score += numbers_to_values[num_counts.most_common()[0][0]]
    score += numbers_to_values[num_counts.most_common()[1][0]]/100

    return score

def check_three_kind(num_counts, numbers_to_values):

    score = 3
    score += numbers_to_values[num_counts.most_common()[0][0]]

    singles = sorted([num_counts.most_common()[1][0],num_counts.most_common()[2][0]], reverse=True)

    score += numbers_to_values[singles[0]]/100
    score += numbers_to_values[singles[1]]/100/100   
    
    return score

def check_two_pair(num_counts, numbers_to_values):
    
    score = 2

    doubles = sorted([num_counts.most_common()[0][0],num_counts.most_common()[1][0]], reverse=True)

    score += numbers_to_values[doubles[0]]
    score += numbers_to_values[doubles[1]]/100

    score += numbers_to_values[num_counts.most_common()[2][0]]/100/100
    
    return score

def check_one_pair(num_counts, numbers_to_values):
    
    score = 1
    score += numbers_to_values[num_counts.most_common()[0][0]]

    singles = sorted([num_counts.most_common()[1][0],num_counts.most_common()[2][0],num_counts.most_common()[3][0]], reverse=True)

    score += numbers_to_values[singles[0]]/100
    score += numbers_to_values[singles[1]]/100/100
    score += numbers_to_values[singles[2]]/100/100/100
    
    return score

def check_high_card(magnitude_num, numbers_to_values):

    score = 0

    singles = sorted(magnitude_num, reverse=True)
    
    score += numbers_to_values[singles[0]]
    score += numbers_to_values[singles[1]]/100
    score += numbers_to_values[singles[2]]/100/100
    score += numbers_to_values[singles[3]]/100/100/100
    score += numbers_to_values[singles[4]]/100/100/100/100 
    
    return score

def check_straight(magnitude_num, numbers_to_values):
    
    score = 0
    
    if (len(np.unique(magnitude_num))==5) & (max(magnitude_num)-min(magnitude_num) == 4):
        
        score = 4
        
        score += numbers_to_values[max(magnitude_num)]
        
    elif (len(np.unique(magnitude_num))==5) & (sorted(magnitude_num, reverse=True)==[14, 5, 4, 3, 2]):
        
        score = 4
        
        score += numbers_to_values[5]
        
    return score
    
def check_flush(suits, magnitude_num, numbers_to_values):
    
    if len(np.unique(suits)) == 1:
        
        score = 5
        
        singles = sorted(magnitude_num, reverse=True)
        
        score += numbers_to_values[singles[0]]
        score += numbers_to_values[singles[1]]/100
        score += numbers_to_values[singles[2]]/100/100
        score += numbers_to_values[singles[3]]/100/100/100
        score += numbers_to_values[singles[4]]/100/100/100/100 
        
    else:
        
        score = 0
        
    return score

def calc_five_card_strength(five_cards):
    
    #straight flush: 8
    #four of a kind: 7
    #full house: 6
    #flush: 5
    #straight: 4
    #trips: 3
    #two pair: 2
    #pair: 1
    #high card: 0
    
    #ones digit is hand type
    #tens-hundreds digits are most important card(s)
    #next two digits are second most imporatant card(s), and so one
    
    score = 0
    
    cards_to_numbers = {"2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "T":10, "J":11, "Q":12, "K":13, "A":14}
    cards_to_values = {"2":.05, "3":.1, "4":.15, "5":.2, "6":.25, "7":.3, "8":.35, "9":.4, "T":.45, "J":.5, "Q":.55, "K":.6, "A":.65}
    numbers_to_values = dict(zip(cards_to_numbers.values(),cards_to_values.values()))
    
    magnitude = [five_cards[i][0] for i in range(len(five_cards))]
    magnitude_num = [cards_to_numbers[magnitude[i]] for i in range(len(magnitude))]
    
    suits = [five_cards[i][1] for i in range(len(five_cards))]
    
    num_counts = collections.Counter(magnitude_num)
    
    #check for quads
    if num_counts.most_common()[0][1] == 4:
        
        score = check_quads(num_counts, numbers_to_values)
    
    #check for full house
    elif (num_counts.most_common()[0][1] == 3) & (num_counts.most_common()[1][1] == 2):
        
        score = check_full_house(num_counts, numbers_to_values)
    
    #check for three of a kind
    elif (num_counts.most_common()[0][1] == 3):
        
        score = check_three_kind(num_counts, numbers_to_values)
    
    #check for two pair
    elif (num_counts.most_common()[0][1] == 2) & (num_counts.most_common()[1][1] == 2):
        
        score = check_two_pair(num_counts, numbers_to_values)
    
    #check for pair
    elif (num_counts.most_common()[0][1] == 2):
        
        score = check_one_pair(num_counts, numbers_to_values)
    
    #check for high card
    else:
        
        score = check_high_card(magnitude_num, numbers_to_values)
    
    straight = check_straight(magnitude_num, numbers_to_values)
    flush = check_flush(suits, magnitude_num, numbers_to_values)
    
    if (flush > 0) & (straight > 0):
        
        score = straight + 4
        
    else:
        
        score = max(score, flush, straight)
    
    return score

def calc_seven_card_strength(hand, board):
    
    all_scores = []
    
    for five_cards in list(itertools.combinations([*hand, *board],5)):
        
        all_scores.append(calc_five_card_strength(list(five_cards)))
        
    return max(all_scores)
    

In [4]:
#MUST RUN
#creates a class to represent a poker board state, with functions that modify itself

class board_state:
    def __init__(self, phase, hand1, hand2, board, mod_deck):
        self.phase = phase
        self.hand1 = hand1
        self.hand2 = hand2
        self.board = board
        self.mod_deck = mod_deck
    
    def show_state(self):
        print('Player 1 hand: ' + str(self.hand1))
        print('Player 2 hand: ' + str(self.hand2))
        print('Board: ' + str(self.board))
    
    def flop(self):
        if self.phase != 0:
            print('error: phase not preflop, cannot execute flop.')
            #error
        elif self.phase == 0:
            self.phase = 1
            self.board = random.sample(self.mod_deck, 3)
            
            self.mod_deck = [self.mod_deck[i] for i in range(len(self.mod_deck)) if self.mod_deck[i] not in self.board]
            
    def turn(self):
        if self.phase != 1:
            print('error: phase not flop, cannot execute turn.')
            #error
        elif self.phase == 1:
            self.phase = 2
            turn = random.sample(self.mod_deck, 1)[0]
            self.board.append(turn)
            
            self.mod_deck.remove(turn)
            
    def river(self):
        if self.phase != 2:
            print('error: phase not turn, cannot execute river.')
            #error
        elif self.phase == 2:
            self.phase = 3
            river = random.sample(self.mod_deck, 1)[0]
            self.board.append(river)
            
            self.mod_deck.remove(river)
            
    def calculate_winner(self):
        
        if self.phase != 3:
            
            print('error: phase not river, cannot calculate winner.')
            
        elif self.phase == 3:
        
            hand1_strength = calc_seven_card_strength(self.board,self.hand1)
            hand2_strength = calc_seven_card_strength(self.board,self.hand2)
            
            if hand1_strength > hand2_strength:

                return 1

            elif hand2_strength > hand1_strength:

                return 2

            else:

                return 0
            
    def all_phases_calculate(self):
        
        self.flop()
        self.turn()
        self.river()
        
        return self.calculate_winner()
        

In [5]:
# MUST RUN
#creates functions that create a board state

def start_game():
    
    hands = random.sample(deck, 4)
    
    hand1 = hands[0:2]
    hand2 = hands[2:4]
    
    mod_deck = [deck[i] for i in range(52) if deck[i] not in hands]
    
    X = board_state(0,hand1,hand2,[],mod_deck)
    
    return X

def start_game_with_hands(hand1, hand2):
    
    mod_deck = [deck[i] for i in range(52) if deck[i] not in [*hand1, *hand2]]
    
    X = board_state(0,hand1,hand2,[],mod_deck)
    
    return X

In [6]:
#MUST RUN
#compares two hands, runs iterations simulations and returns the number of times hand 1 won, hand 2 won, and ties.
#If equity is True, ignores ties

def simulate_hand_fight(hand1, hand2, iterations=1000, equity=True):
    
    results = [0,0,0]
    
    for trial in range(iterations):
        
        game_trial = start_game_with_hands(hand1, hand2)
        
        results[game_trial.all_phases_calculate()] += 1
        
    if equity==True:
        
        results = [results[1]+results[0]/2, results[2]+results[0]/2]
        
    return results

In [7]:
#creates matrix of handfight win probabilities of hands pitted against each other (52*51/2) by (52*51/2)

def create_pf_win_prob_matrix(iters):
    
    num_hands = int(52*51/2)
    
    #converts tuples to lists
    poss_hands = [list(list(itertools.combinations(deck,2))[i]) for i in range(num_hands)]
    hand_nums = np.array(range(num_hands))
    num_to_hand = dict(zip(hand_nums, poss_hands))
    
    surplus_win_probs = np.zeros([num_hands,num_hands])
    
    for hand1_num in tqdm(range(num_hands)):
        
        for hand2_num in tqdm(range(hand1_num)):
            
            hand1 = num_to_hand[hand1_num]
            hand2 = num_to_hand[hand2_num]
            
            surplus_win_prob_p1 = (2*simulate_hand_fight(hand1,hand2, iters, True)[0]/iters-1)
            surplus_win_probs[hand1_num,hand2_num] = surplus_win_prob_p1
            surplus_win_probs[hand2_num,hand1_num] = -surplus_win_prob_p1
    
    return surplus_win_probs

In [11]:
#very complicated code that returns regret for both players after many iterations
#this is what we have to fix (for this specific sim) - i don't know if it's broken though lol
#we assume blocker effects don't exist here

def hu_pf_sim_v2(surplus_win_probs, num_iter = 10000, iters=100):
    
    #set overall parameters
    num_hands = int(52*51/2)
    stacks = 10 
    
    #create general variables/dicts needed later
    poss_hands = [list(list(itertools.combinations(deck,2))[i]) for i in range(num_hands)]
    hand_nums = np.array(range(num_hands))
    num_to_hand = dict(zip(hand_nums, poss_hands))
    

    #create regret matrices; first element is fold, second is shove. We start with both equal to 1 such that each action is 50%
    p1_strats = np.array([[1, 1]] * num_hands).astype(float) #fold, shove
    p2_strats = np.array([[1, 1]] * num_hands).astype(float) #fold, call (conditional on p1 shoving)
    
    #the following code is the optimization and runs num_iter times
    for iter in tqdm(range(num_iter)):
        
        hand1_num, hand2_num = random.sample(list(hand_nums),2)
        hand1 = num_to_hand[hand1_num]
        hand2 = num_to_hand[hand2_num]
        
        #an information set is one hand. we calculate win probability based on p1's hand against all possible hands p2 has, conditional on p2 calling
        #for now, IGNORE BLOCKER EFFECTS
        all_win_prob_p1 = 0
        total_prob_p2_calls = 0
        for hand_i in range(num_hands):
            per_hand_win_prob_p1 = (surplus_win_probs[hand1_num, hand_i]+1)/2
            
            #calc prob p1 bets
            if p2_strats[hand_i, 0]<=0:
                if p2_strats[hand_i, 1]<=0:
                    p2_sim_prob_fold=.5
                else:
                    p2_sim_prob_fold=0
            else:
                if p2_strats[hand_i, 1]<=0:
                    p2_sim_prob_fold=1
                else:
                    p2_sim_prob_fold = p2_strats[hand_i, 0]/sum(p2_strats[hand_i, :])
            
            all_win_prob_p1 = all_win_prob_p1 + (1-p2_sim_prob_fold)*per_hand_win_prob_p1
            total_prob_p2_calls = total_prob_p2_calls + (1-p2_sim_prob_fold)
        
        win_prob_p1 = all_win_prob_p1/total_prob_p2_calls
        
        #this code chunk calculates the expected win probability for player 2 given only their hand. we must condition on whether p1 will bet or not, so given that we face a bet, find our expected WP.
        all_win_prob_p2 = 0
        total_prob_p1_bets = 0
        for hand_i in range(num_hands):
            per_hand_win_prob_p2 = 1-(surplus_win_probs[hand_i, hand2_num]+1)/2
            
            #calc prob p1 bets
            if p1_strats[hand_i, 0]<=0:
                if p1_strats[hand_i, 1]<=0:
                    p1_sim_prob_fold=.5
                else:
                    p1_sim_prob_fold=0
            else:
                if p1_strats[hand_i, 1]<=0:
                    p1_sim_prob_fold=1
                else:
                    p1_sim_prob_fold = p1_strats[hand_i, 0]/sum(p1_strats[hand_i, :])
            
            all_win_prob_p2 = all_win_prob_p2 + (1-p1_sim_prob_fold)*per_hand_win_prob_p2
            total_prob_p1_bets = total_prob_p1_bets + (1-p1_sim_prob_fold)
        
        win_prob_p2 = all_win_prob_p2/total_prob_p1_bets
            
        #we now calculate, based on the regret matrices:
        #from p2's perspective, what is the probability that p1 folds
        #form p1's perspective, what is the probability that p2 folds
        #sim is your guess of the other person's action
        #con is your probability of action
        p1_sim_prob_fold = 0
        p2_sim_prob_fold = 0
        
        for hand_i in range(num_hands):

            if p1_strats[hand_i, 0]<=0:
                if p1_strats[hand_i, 1]<=0:
                    p1_sim_prob_fold+=.5
                else:
                    p1_sim_prob_fold+=0
            else:
                if p1_strats[hand_i, 1]<=0:
                    p1_sim_prob_fold+=1
                else:
                    p1_sim_prob_fold += p1_strats[hand_i, 0]/sum(p1_strats[hand_i, :])
                    
                    
            if p2_strats[hand_i, 0]<=0:
                if p2_strats[hand_i, 1]<=0:
                    p2_sim_prob_fold+=.5
                else:
                    p2_sim_prob_fold+=0
            else:
                if p2_strats[hand_i, 1]<=0:
                    p2_sim_prob_fold+=1
                else:
                    p2_sim_prob_fold += p2_strats[hand_i, 0]/sum(p2_strats[hand_i, :])
                    
        p1_sim_prob_fold = p1_sim_prob_fold/(num_hands)
        p2_sim_prob_fold = p2_sim_prob_fold/(num_hands)
        
        if p2_strats[hand2_num, 0]<=0:
            if p2_strats[hand2_num, 1]<=0:
                p2_con_prob_fold=.5
            else:
                p2_con_prob_fold=0
        else:
            if p2_strats[hand2_num, 1]<=0:
                p2_con_prob_fold=1
            else:
                p2_con_prob_fold = p2_strats[hand2_num, 0]/sum(p2_strats[hand2_num, :])
              
        if p1_strats[hand1_num, 0]<=0:
            if p1_strats[hand1_num, 1]<=0:
                p1_con_prob_fold=.5
            else:
                p1_con_prob_fold=0
        else:
            if p1_strats[hand1_num, 1]<=0:
                p1_con_prob_fold=1
            else:
                p1_con_prob_fold = p1_strats[hand1_num, 0]/sum(p1_strats[hand1_num, :])
        
        #calculate actual payouts for folding and pushing
        p1_actual_payouts = np.array([0, 1.5*p2_sim_prob_fold + ((stacks+.5)*win_prob_p1-(stacks-.5)*(1-win_prob_p1))*(1-p2_sim_prob_fold)])
        p1_mixed_chosen_payout = np.dot([p1_con_prob_fold,(1-p1_con_prob_fold)],p1_actual_payouts)
        p1_regret = p1_actual_payouts-p1_mixed_chosen_payout
        
        #calculate actual payouts for folding and calling
        p2_actual_payouts = np.array([0, ((stacks+1)*(win_prob_p2)-(stacks-1)*(1-win_prob_p2))])*(1-p1_sim_prob_fold)
        p2_mixed_chosen_payout = np.dot([p2_sim_prob_fold,(1-p2_sim_prob_fold)],p2_actual_payouts)
        p2_regret = p2_actual_payouts-p2_mixed_chosen_payout
        
        p1_strats[hand1_num] = (p1_strats[hand1_num] + p1_regret)
        p2_strats[hand2_num] = (p2_strats[hand2_num] + p2_regret)
        
    return p1_strats, p2_strats

In [9]:
#create very approximated win prob matrix

surplus_win_probs_1 = create_pf_win_prob_matrix(1)


  0%|          | 0/1326 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/1321 [00:00<?, ?it/s]

  0%|          | 0/1322 [00:00<?, ?it/s]

  0%|          | 0/1323 [00:00<?, ?it/s]

  0%|          | 0/1324 [00:00<?, ?it/s]

  0%|          | 0/1325 [00:00<?, ?it/s]

In [43]:
surplus_win_probs_10 = pd.read_csv('10_sim_hand_fight.csv', header=None).values

In [ ]:
#find regrets per player

[p1_resp, p2_resp] = hu_pf_sim_v2(surplus_win_probs_10,100000)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [14]:
#view regrets for each hand

num_hands = int(52*51/2)
poss_hands = [list(list(itertools.combinations(deck,2))[i]) for i in range(num_hands)]
new_array = [[p1_resp[i],poss_hands[i]] for i in range(len(poss_hands))]
new_array

[[array([ 1.65619166, -9.12821425]), ['2s', '3s']],
 [array([-11.78806892,   1.70761894]), ['2s', '4s']],
 [array([ 1.5856194, -5.5965834]), ['2s', '5s']],
 [array([ 1.56914   , -0.03074769]), ['2s', '6s']],
 [array([-1.59925378,  1.64607295]), ['2s', '7s']],
 [array([1.27280459, 0.8629446 ]), ['2s', '8s']],
 [array([  1.59747737, -20.34617939]), ['2s', '9s']],
 [array([  1.5465407 , -10.36713931]), ['2s', 'Ts']],
 [array([-29.75517473,   1.47390985]), ['2s', 'Js']],
 [array([-5.75769504,  1.58192276]), ['2s', 'Qs']],
 [array([-7.32223134,  1.74534325]), ['2s', 'Ks']],
 [array([-60.04893854,   1.87183769]), ['2s', 'As']],
 [array([-43.76037803,   1.65986204]), ['2s', '2h']],
 [array([  1.9012389 , -50.07925669]), ['2s', '3h']],
 [array([  1.71667248, -45.44735697]), ['2s', '4h']],
 [array([  1.66434804, -20.10657204]), ['2s', '5h']],
 [array([  1.87312695, -43.41742311]), ['2s', '6h']],
 [array([  1.88021712, -47.64763895]), ['2s', '7h']],
 [array([  1.68230268, -20.27204254]), ['2s', 